# usage data check

In [7]:
from sqlalchemy import create_engine, types
from datetime import datetime, timedelta
from IPython.display import display
import matplotlib.pyplot as plt
import reliant_scrape as rs
import mysql.connector
import pandas as pd
import numpy as np
import pprint
import yaml
import boto3
import boto
import json
import sys
import os

base = os.path.join('C:\\', 'Users', 'steve', 'Documents', 'reliant-scrape')
data = os.path.join(base, 'data')
pp = pprint.PrettyPrinter(indent = 1)
os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1'

with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader = yaml.FullLoader)
        
with open('db_creds.json', 'r') as f:
    db_creds = json.load(f)
    
print('loaded credentials & imported modules.')

loaded credentials & imported modules.


## query

In [8]:
result = rs.mysql_query("""SELECT * FROM reliant_energy_db.daily_use""", db_creds)

## data

In [9]:
result

,Date,Usage (kWh),Cost ($),Hi,Low
0,2020-08-25 00:00:00,1.1,0.13,94,79
1,2020-08-25 01:00:00,1.2,0.14,94,79
2,2020-08-25 02:00:00,1.0,0.12,94,79
3,2020-08-25 03:00:00,1.0,0.12,94,79
4,2020-08-25 04:00:00,1.5,0.18,94,79
...,...,...,...,...,...
1435,2020-11-02 19:00:00,1.8,0.23,72,51
1436,2020-11-02 20:00:00,0.9,0.11,72,51
1437,2020-11-02 21:00:00,0.7,0.09,72,51
1438,2020-11-02 22:00:00,0.7,0.08,72,51


In [10]:
result.shape

(1440, 5)

In [13]:
print('data ranges from {} to {}.'.format(min(result.Date), max(result.Date)))

data ranges from 2020-08-25 00:00:00 to 2020-11-02 23:00:00.


## kusto database

In [18]:
#! conda install -c conda-forge pyodbc -y

In [19]:
#! pip install azure-kusto-data azure-kusto-ingest

In [22]:
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties

from azure.kusto.ingest import DataFormat, IngestionProperties, KustoStreamingIngestClient, ReportLevel
from azure.kusto.ingest.status import KustoIngestStatusQueues
print('imported additional modules.')

imported additional modules.


## query kusto

In [24]:
with open('creds-kusto.json', 'r') as f:
    creds = json.load(f)
    pp.pprint(creds)

FileNotFoundError: [Errno 2] No such file or directory: 'creds-kusto.json'

In [ ]:
KCSB = KustoConnectionStringBuilder.with_aad_application_key_authentication(kusto_creds['cluster_uri'], 
                                                                            kusto_creds['client_id'], 
                                                                            kusto_creds['client_secret'], 
                                                                            kusto_creds['tenant_id'])

kusto_client = KustoClient(KCSB)
kusto_ingest_client = KustoStreamingIngestClient(KCSB)
    
database = "sourceWellDatabase"

start = time.time()
try:
    response = kusto_client.execute(database, query)
    end = time.time()
    print('executed query in {} seconds.'.format(round(end-start,4)))
    dataframe = dataframe_from_result_table(response.primary_results[0])
    return(dataframe)

except:
    print('error.')